## Importação de bibliotecas utilizadas

In [1]:
import numpy as np
import pandas as pd

## Faz o carregamento da base e aplica limpezas

In [37]:
# Carrega a base e avalia necessidade de limpeza de campos numéricos
dataset = pd.read_csv("../data/dataset.csv").dropna()
campos_numericos = [
    "Income", "MntWines", "MntFruits", "MntMeatProducts",
    "MntFishProducts", "MntSweetProducts", "MntGoldProds",
    "NumDealsPurchases", "NumWebPurchases", "NumCatalogPurchases",
    "NumStorePurchases", "NumWebVisitsMonth", "Recency"
]

# Para cada campo, avalia a diferença entre o máximo e q3,
# decidindo a necessidade de aplicar método de limpeza.
# Avaliando o conjunto, a data de nascimento passará por
# um processo diferente.
para_limpeza = {
    "Year_Birth": {"lim_inf": 1930, "lim_sup": 2000}
}

for campo in campos_numericos:
    maximo = max(dataset[campo].values)
    q3 = np.percentile(dataset[campo].values, 75)
    dif = (maximo - q3) / q3
    
    if dif > 0.4:
        # Cadastra o campo para remoção de outliers
        q1 = np.percentile(dataset[campo].values, 25)
        para_limpeza[campo] = {
            "lim_inf": q1 - 1.5 * (q3 - q1),
            "lim_sup": q3 + 1.5 * (q3 - q1)
        }
        
# Remove os outliers para cada campo
novo_dataset = []
for i in range(0, len(dataset)):
    valido = True
    for campo in para_limpeza.keys():
        valor = dataset[campo].values[i]
        
        # Se o valor do campo for outlier, registra como skip
        if valor < para_limpeza[campo]["lim_inf"] or\
            valor > para_limpeza[campo]["lim_sup"]:
            valido = False
            break
            
    if valido:
        novo_dataset.append(dataset.values[i])
        
# Finaliza a limpeza
dataset = pd.DataFrame(novo_dataset, columns=dataset.columns)
dataset.to_csv("../data/dataset_clean.csv")